## Coursera Capstone Project

## Third Assignment

In [59]:
!pip install BeautifulSoup4
!pip install geopy
import pandas as pd
import numpy as np
import requests
import folium 
import matplotlib.cm as cm
import matplotlib.colors as colors
import json

from urllib.request import urlopen
from geopy.geocoders import Nominatim 
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


#### Build the code to scrape the following Wikipedia page

In [60]:
source = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = urlopen(source).read().decode('utf-8')
soup = BeautifulSoup(page, 'html.parser')

In [61]:
table = soup.table.tbody

def table_cell(i):
    cells = i.find_all('td')
    row = []
    
    for cell in cells:
        if cell.a:            
            if (cell.a.text):
                row.append(cell.a.text)
                continue
        row.append(cell.string.strip())
        
    return row

def table_row():    
    data = []  
    
    for tr in table.find_all('tr'):
        row = table_cell(tr)
        if len(row) != 3:
            continue
        data.append(row)        
    
    return data

In [62]:
data = table_row()
columns = ['PostalCode', 'Borough', 'Neighborhood']
df_Toronto = pd.DataFrame(data, columns = columns)
df_Toronto.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [63]:
df_Tor = df_Toronto[df_Toronto.Borough != 'Not assigned']
df_Tor = df_Tor.sort_values(by=['PostalCode','Borough'])
df_Tor.reset_index(inplace = True)
df_Tor.drop('index',axis = 1, inplace = True)
df_Tor.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [64]:
df_Tor.shape

(103, 3)

#### Get the latitude and the longitude coordinates of each Postal Code

In [65]:
latit_longi = pd.read_csv('https://cocl.us/Geospatial_data')

In [66]:
latit_longi.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [67]:
df_Tor = df_Tor.join(latit_longi.set_index('Postal Code'), on = 'PostalCode')
df_Tor.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


#### Explore and cluster the neighborhoods in Toronto and then replicate the same analysis we did to the New York City
#### Get the latitude and longitude values of Toronto

In [68]:
address = 'Toronto, Canada'
geolocator = Nominatim(user_agent = 'Toronto_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [69]:
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(df_Tor['Latitude'], df_Tor['Longitude'], df_Tor['Borough'], df_Tor['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

#### Work with only boroughs that contain the word Toronto

In [70]:
Toronto_data = df_Tor[df_Tor['Borough'].str.contains('Downtown Toronto')].reset_index(drop=True)
Toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [71]:
map_Toronto_data = folium.Map(location=[latitude, longitude], zoom_start = 12)
for lat, lng, borough, neighborhood in zip(
        Toronto_data['Latitude'], 
        Toronto_data['Longitude'], 
        Toronto_data['Borough'], 
        Toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto_data)  

map_Toronto_data

#### Define Foursquare Credentials and Version

In [108]:
CLIENT_ID = 'Y53TUHWVOZ4I4Z2JA5U2Y5ZUSYWRG1Y5MNHSQ2NGYVKNRCL0' 
CLIENT_SECRET = 'EIN5XIXWZZQPVXGPE2UCOKN3ZY3KBDSUICZ05NNXEJKXJTBS' 
VERSION = '20180605' 
LIMIT = 100 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Y53TUHWVOZ4I4Z2JA5U2Y5ZUSYWRG1Y5MNHSQ2NGYVKNRCL0
CLIENT_SECRET:EIN5XIXWZZQPVXGPE2UCOKN3ZY3KBDSUICZ05NNXEJKXJTBS


#### Get the neighborhood's name.

In [109]:
name_neighbor = Toronto_data.loc[0, 'Neighborhood']
print(f"The first neighborhood's name is '{name_neighbor}'.")

The first neighborhood's name is 'Rosedale'.


#### Get the neighborhood's latitude and longitude values

In [110]:
neighborhood_latitude = Toronto_data.loc[0, 'Latitude'] 
neighborhood_longitude = Toronto_data.loc[0, 'Longitude'] 

print('Latitude and longitude values of {} are {}, {}.'.format(name_neighbor, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rosedale are 43.6795626, -79.37752940000001.


#### Top 100 venues that are in Rosedale within a radius of 700 meters

In [111]:
LIMIT = 100 
radius = 700 

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url 

'https://api.foursquare.com/v2/venues/explore?&client_id=Y53TUHWVOZ4I4Z2JA5U2Y5ZUSYWRG1Y5MNHSQ2NGYVKNRCL0&client_secret=EIN5XIXWZZQPVXGPE2UCOKN3ZY3KBDSUICZ05NNXEJKXJTBS&v=20180605&ll=43.6795626,-79.37752940000001&radius=700&limit=100'

In [112]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fd4f3ac2e172d2de0f309ff'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Rosedale',
  'headerFullLocation': 'Rosedale, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 5,
  'suggestedBounds': {'ne': {'lat': 43.685862606300006,
    'lng': -79.36883453584119},
   'sw': {'lat': 43.67326259369999, 'lng': -79.38622426415884}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c0960fb009a0f472fabe7bf',
       'name': 'Craigleigh Gardens',
       'location': {'address': '160 South Drive',
        'crossStreet': 'at Elm Ave',
        'lat': 43.67809940868806,
        'lng': -79.37158584594727,
        'labeledLatLngs': [{'lab

In [113]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [114]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) 

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,name,categories,lat,lng
0,Craigleigh Gardens,Park,43.678099,-79.371586
1,Rosedale Park,Playground,43.682328,-79.378934
2,Whitney Park,Park,43.682036,-79.373788
3,Alex Murray Parkette,Park,43.678300,-79.382773
4,Milkman's Lane,Trail,43.676352,-79.373842


In [115]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

5 venues were returned by Foursquare.


#### Explore Neighborhoods in Toronto

In [116]:
def getNearbyVenues(names, latitudes, longitudes, radius = 700):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
     
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)   

In [117]:
Toronto_venues = getNearbyVenues(names = Toronto_data['Neighborhood'],
                                   latitudes = Toronto_data['Latitude'],
                                   longitudes = Toronto_data['Longitude']
                                  )

Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Queen's Park, Ontario Provincial Government


In [118]:
print(Toronto_venues.shape)
Toronto_venues.head()

(1561, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Craigleigh Gardens,43.678099,-79.371586,Park
1,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
2,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
3,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
4,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail


#### How many venues were returned for each neighborhood

In [119]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,100,100,100,100,100,100
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",26,26,26,26,26,26
Central Bay Street,100,100,100,100,100,100
Christie,25,25,25,25,25,25
Church and Wellesley,100,100,100,100,100,100
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
"First Canadian Place, Underground city",100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100
"Harbourfront East, Union Station, Toronto Islands",99,99,99,99,99,99


#### How many unique categories can be curated from all the returned venues

In [120]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 221 uniques categories.


#### Analyze Each Neighborhood

In [121]:
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]
Toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Animal Shelter,...,Tunnel,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [122]:
Toronto_onehot.shape

(1561, 221)

#### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [123]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Tunnel,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint
0,Berczy Park,0.010000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.00,0.00,0.010000,0.00,0.00,0.00,0.000000,0.000000,0.00
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.00,0.038462,0.038462,0.038462,0.076923,0.076923,0.076923,0.00,...,0.038462,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.00
2,Central Bay Street,0.010000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.00,0.01,0.010000,0.01,0.00,0.01,0.010000,0.000000,0.00
3,Christie,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.00
4,Church and Wellesley,0.020000,0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.01,...,0.000000,0.00,0.00,0.000000,0.00,0.01,0.01,0.000000,0.000000,0.00
5,"Commerce Court, Victoria Hotel",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.03,...,0.000000,0.00,0.00,0.020000,0.00,0.00,0.00,0.010000,0.000000,0.00
6,"First Canadian Place, Underground city",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.03,...,0.000000,0.00,0.00,0.010000,0.00,0.00,0.00,0.010000,0.000000,0.00
7,"Garden District, Ryerson",0.010000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.01,...,0.000000,0.00,0.00,0.000000,0.00,0.00,0.00,0.010000,0.000000,0.00
8,"Harbourfront East, Union Station, Toronto Islands",0.010101,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.00,0.00,0.010101,0.00,0.00,0.00,0.000000,0.010101,0.00
9,"Kensington Market, Chinatown, Grange Park",0.020000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.01,0.00,0.060000,0.00,0.00,0.02,0.010000,0.000000,0.00


In [124]:
Toronto_grouped.shape

(19, 221)

#### Print each neighborhood along with the top 5 most common venues

In [125]:
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending = False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
         venue  freq
0  Coffee Shop  0.10
1        Hotel  0.06
2     Beer Bar  0.04
3         Café  0.04
4   Restaurant  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                 venue  freq
0  Rental Car Location  0.12
1      Harbor / Marina  0.08
2        Boat or Ferry  0.08
3       Airport Lounge  0.08
4      Airport Service  0.08


----Central Bay Street----
                 venue  freq
0          Coffee Shop  0.14
1                 Café  0.06
2          Art Gallery  0.04
3  Japanese Restaurant  0.03
4        Movie Theater  0.02


----Christie----
                venue  freq
0       Grocery Store  0.20
1                Café  0.16
2                Park  0.12
3         Coffee Shop  0.08
4  Italian Restaurant  0.04


----Church and Wellesley----
                 venue  freq
0          Coffee Shop  0.10
1  Japanese Restaurant  0.05
2           Restaurant  0.04
3                 Café  0.

In [126]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending = False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Display the top 7 venues for each neighborhood.

In [127]:
num_top_venues = 7

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns = columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
0,Berczy Park,Coffee Shop,Hotel,Restaurant,Café,Beer Bar,Japanese Restaurant,Park
1,"CN Tower, King and Spadina, Railway Lands, Har...",Rental Car Location,Boat or Ferry,Harbor / Marina,Sculpture Garden,Airport Lounge,Airport Service,Airport Terminal
2,Central Bay Street,Coffee Shop,Café,Art Gallery,Japanese Restaurant,Italian Restaurant,Sandwich Place,Ramen Restaurant
3,Christie,Grocery Store,Café,Park,Coffee Shop,Athletics & Sports,Italian Restaurant,Nightclub
4,Church and Wellesley,Coffee Shop,Japanese Restaurant,Restaurant,Café,Gay Bar,Sushi Restaurant,Yoga Studio


#### Cluster Neighborhoods

In [128]:
kclusters = 5
Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters = kclusters, random_state=0).fit(Toronto_grouped_clustering)
kmeans.labels_[0:10] 

array([0, 3, 2, 4, 2, 0, 0, 2, 0, 2], dtype=int32)

#### New dataframe that includes the cluster as well as the top 7 venues for each neighborhood.

In [129]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = Toronto_data
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
Toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,1,Park,Playground,Trail,Wings Joint,Dessert Shop,Electronics Store,Eastern European Restaurant
1,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675,2,Coffee Shop,Pizza Place,Restaurant,Café,Grocery Store,Park,Diner
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,2,Coffee Shop,Japanese Restaurant,Restaurant,Café,Gay Bar,Sushi Restaurant,Yoga Studio
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2,Coffee Shop,Park,Restaurant,Pub,Theater,Bakery,Café
4,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,2,Coffee Shop,Hotel,Gastropub,Japanese Restaurant,Sushi Restaurant,Electronics Store,Plaza


In [130]:
map_clusters_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters_Toronto)
       
map_clusters_Toronto

#### Examine Clusters

##### Cluster 1

In [131]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
5,Downtown Toronto,0,Coffee Shop,Café,Gastropub,Restaurant,Seafood Restaurant,Bakery,Clothing Store
6,Downtown Toronto,0,Coffee Shop,Hotel,Restaurant,Café,Beer Bar,Japanese Restaurant,Park
8,Downtown Toronto,0,Coffee Shop,Café,Hotel,Restaurant,Theater,Bar,Clothing Store
9,Downtown Toronto,0,Coffee Shop,Hotel,Boat or Ferry,Restaurant,Brewery,Baseball Stadium,Plaza
10,Downtown Toronto,0,Hotel,Coffee Shop,Café,Restaurant,American Restaurant,Gym,Seafood Restaurant
11,Downtown Toronto,0,Coffee Shop,Hotel,Café,Beer Bar,Gastropub,Restaurant,Japanese Restaurant
15,Downtown Toronto,0,Coffee Shop,Restaurant,Japanese Restaurant,Café,Hotel,Beer Bar,Gym
16,Downtown Toronto,0,Hotel,Coffee Shop,Café,Seafood Restaurant,Restaurant,Japanese Restaurant,Theater


##### Cluster 2

In [132]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
0,Downtown Toronto,1,Park,Playground,Trail,Wings Joint,Dessert Shop,Electronics Store,Eastern European Restaurant


##### Cluster 3

In [133]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
1,Downtown Toronto,2,Coffee Shop,Pizza Place,Restaurant,Café,Grocery Store,Park,Diner
2,Downtown Toronto,2,Coffee Shop,Japanese Restaurant,Restaurant,Café,Gay Bar,Sushi Restaurant,Yoga Studio
3,Downtown Toronto,2,Coffee Shop,Park,Restaurant,Pub,Theater,Bakery,Café
4,Downtown Toronto,2,Coffee Shop,Hotel,Gastropub,Japanese Restaurant,Sushi Restaurant,Electronics Store,Plaza
7,Downtown Toronto,2,Coffee Shop,Café,Art Gallery,Japanese Restaurant,Italian Restaurant,Sandwich Place,Ramen Restaurant
12,Downtown Toronto,2,Café,Coffee Shop,Pizza Place,Bubble Tea Shop,Bakery,Bar,Sandwich Place
13,Downtown Toronto,2,Bar,Café,Vegetarian / Vegan Restaurant,Mexican Restaurant,Coffee Shop,Park,Yoga Studio
18,Downtown Toronto,2,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Park,Yoga Studio,Middle Eastern Restaurant


##### Cluster 4

In [134]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
14,Downtown Toronto,3,Rental Car Location,Boat or Ferry,Harbor / Marina,Sculpture Garden,Airport Lounge,Airport Service,Airport Terminal


##### Cluster 5

In [135]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
17,Downtown Toronto,4,Grocery Store,Café,Park,Coffee Shop,Athletics & Sports,Italian Restaurant,Nightclub
